In [1]:
%pip install openai
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
import re
import numpy as np
import pandas as pd
import json
import os
import subprocess

In [3]:

def extract_voltages(text):
    # Regular expression to match the voltage lines
    voltage_pattern = re.compile(r'V\((\d+)\)\s+([\d\.e\+\-]+)')
    
    # Find all matches in the text
    voltages = voltage_pattern.findall(text)
    
    # Convert matches to a dictionary
    voltage_dict = {f'V({node})': float(voltage) for node, voltage in voltages}
    
    return voltage_dict

def spice_sim(data):
    # Write the SPICE content to a file
    with open("circuit.cir", "w") as file:
        file.write(data["spice"])

    # Run ngspice
    result = subprocess.run(["C:\\Users\\denis\\ngspice-42_64\\Spice64\\bin\\ngspice_con", "-b", "circuit.cir"], capture_output=True, text=True)
    voltages_sim = extract_voltages(result.stdout)
    return voltages_sim

In [4]:
# set your openai api key here
os.environ['OPENAI_API_KEY']=''

In [5]:
datagen_model = "gpt-4o"
client = OpenAI()

In [6]:
examples = [
    {
        "description": "Series RC circuit with DC voltage source.",
        "spice": """
* Series RC circuit
V1 1 0 DC 10
R1 1 2 1k
C1 2 0 1u
.op
.print v(1) v(2)
.end
""",
        "voltages": {
            "1": 10,
            "2": 9.9
        }
    },
{
  "description": "Series RL circuit with DC current source.",
        "spice": """
* RL Circuit with Current Source
I1 0 1 DC 2m
R1 1 2 100
L1 2 0 1m
.op
.print v(1) v(2)
.end
""",
        "voltages": {
            "1": 10,
            "2": 9.9
        }
    },
{
  "description": "Series RLC circuit with DC voltage source.",
        "spice": """
* RLC Circuit with Voltage Source
V1 1 0 DC 5
R1 1 2 500
L1 2 3 100m
C1 3 0 1u
.op
.print v(1) v(2) v(3)
.end
""",
        "voltages": {
            "1": 10,
            "2": 9.9
        }
    },
{
  "description": "Parallel RLC circuit with DC voltage source.",
        "spice": """
* Parallel RLC Circuit with Voltage Source
V1 1 0 DC 12
R1 1 0 2k
L1 1 0 50m
C1 1 0 2u
.op
.print v(1) v(2)
.end
""",
        "voltages": {
            "1": 10,
            "2": 9.9
        }
    },
{
  "description": "Series RLC circuit with DC current source.",
        "spice": """
* Series RLC Circuit with Current Source
I1 0 1 DC 1m
R1 1 2 10k
L1 2 3 10m
C1 3 0 10n
.op
.print v(1) v(2) v(3)
.end

""",
        "voltages": {
            "1": 10,
            "2": 9.9
        }
    },
]

In [7]:
synthetic_examples = []

In [20]:
for i in examples:
    volts = spice_sim(i)
    i["voltages"] = volts

print(examples)

[{'description': 'Series RC circuit with DC voltage source.', 'spice': '\n* Series RC circuit\nV1 1 0 DC 10\nR1 1 2 1k\nC1 2 0 1u\n.op\n.print v(1) v(2)\n.end\n', 'voltages': {'V(2)': 10.0, 'V(1)': 10.0}}, {'description': 'Series RL circuit with DC current source.', 'spice': '\n* RL Circuit with Current Source\nI1 0 1 DC 2m\nR1 1 2 100\nL1 2 0 1m\n.op\n.print v(1) v(2)\n.end\n', 'voltages': {'V(2)': 0.0, 'V(1)': 0.2}}, {'description': 'Series RLC circuit with DC voltage source.', 'spice': '\n* RLC Circuit with Voltage Source\nV1 1 0 DC 5\nR1 1 2 500\nL1 2 3 100m\nC1 3 0 1u\n.op\n.print v(1) v(2) v(3)\n.end\n', 'voltages': {'V(3)': 5.0, 'V(2)': 5.0, 'V(1)': 5.0}}, {'description': 'Parallel RLC circuit with DC voltage source.', 'spice': '\n* Parallel RLC Circuit with Voltage Source\nV1 1 0 DC 12\nR1 1 2 2k\nL1 1 2 50m\nC1 1 2 2u\n.op\n.print v(1) v(2)\n.end\n', 'voltages': {'V(2)': 12.0, 'V(1)': 12.0}}, {'description': 'Series RLC circuit with DC current source.', 'spice': '\n* Series RL

In [9]:
import requests
import random

# Define the URL of the word list
url = 'https://www.mit.edu/~ecprice/wordlist.10000'

# Send a GET request to the URL
response = requests.get(url)

# Get the content of the response
word_list = response.text.splitlines()

    

In [10]:
def generate_messages(examples, synthetic_examples):
    messages = [
        {
            "role": "system",
            "content": ("You are a helpful assistant designed to generate synthetic data. "
                        "Simulation Program with Integrated Circuit Emphasis (SPICE) is an essential software tool for electronics design that enables engineers to model and analyze the behavior of electronic circuits before they are physically constructed. "
                        "The user will provide a description of the circuit, and you need to respond with the SPICE code based on the description and the voltages derived from the SPICE simulation. "
                        "Explain your thought process for generating the SPICE code based on the description and the calculation process for getting the voltages."
                        "Provide the response in the following JSON format:\n"
                        "{\n"
                        "  \"description\": \"<description>\",\n"
                        " \"translation thought\": \"<translating to spice>\",\n"
                        "  \"spice\": \"<SPICE code>\",\n"
                        " \"calculation thought\": \"<computing the voltages>\",\n"
                        "  \"voltages\": <voltages>\n"
                        "}\n"
                        )
        }
    ]
    for example in examples:
        messages.append({"role": "user", "content": f"Description: {example['description']}"})
        messages.append({
            "role": "assistant",
            "content": json.dumps({
                "description": example['description'],
                "spice": example['spice'],
                "voltages": example['voltages']
            }, indent=4)
        })
    for example in synthetic_examples:
        messages.append({"role": "user", "content": f"Description: {example['description']}"})
        messages.append({
            "role": "assistant",
            "content": json.dumps({
                "description": example['description'],
                "translation thought": example['translation thought'],
                "spice": example['spice'],
                "calculation thought": example['calculation thought'],
                "voltages": example['voltages']
            }, indent=4)
        })
    messages.append({"role": "user", "content": f"Generate new uique simple DC passive synthetic circuit data with varied structure and number of components {random.choice(word_list)} {random.choice(word_list)}."})
    return messages

In [11]:
def generate_synthetic_data(examples, synthetic_examples):
    messages = generate_messages(examples, synthetic_examples)
    response = client.chat.completions.create(
        model=datagen_model,
        messages=messages,
        max_tokens=500,
        n=5,
        stop=None,
        temperature=0.7,
        response_format={"type": "json_object"}
    )
    return response

In [12]:
epoch = 0
while len(synthetic_examples) <= 50:
    epoch += 1
    synthetic_data = generate_synthetic_data(examples, synthetic_examples)
    print("Generated Synthetic Data:")

    count = 0
    for i in synthetic_data.choices:
        try:
            data = json.loads(i.message.content)
            print(data["voltages"])
            volts = spice_sim(data)
            print(volts)
        
            err = sum((volts[j] - data["voltages"][j])**2 for j in volts)
            if err <= 0.1:
                synthetic_examples.append(data)
                count += 1
        except:
            continue
    
    print(len(synthetic_examples), epoch, count/len(synthetic_data.choices))


Generated Synthetic Data:
{'V(3)': 15.0, 'V(2)': 5.0, 'V(1)': 15.0}
{'V(3)': 0.0, 'V(2)': 15.0, 'V(1)': 15.0}
{'V(3)': 10.0, 'V(2)': 10.0, 'V(1)': 15.0}
{'V(3)': 15.0, 'V(2)': 15.0, 'V(1)': 15.0}
{'V(3)': 5.0, 'V(2)': 10.0, 'V(1)': 15.0}
{'V(3)': 15.0, 'V(2)': 15.0, 'V(1)': 15.0}
{'V(2)': 15.0, 'V(1)': 15.0}
{'V(2)': 15.0, 'V(1)': 15.0}
{'V(3)': 15.0, 'V(2)': 11.25, 'V(1)': 15.0}
{'V(3)': 15.0, 'V(2)': 15.0, 'V(1)': 15.0}
1 1 0.2
Generated Synthetic Data:
{'V(3)': 15.0, 'V(2)': 5.0, 'V(1)': 5.0}
{'V(3)': 0.0, 'V(2)': 15.0, 'V(1)': 5.0}
{'V(3)': 20.0, 'V(2)': 20.0, 'V(1)': 20.0}
{'V(3)': 15.38462, 'V(2)': 15.38462, 'V(1)': 20.0}
{'V(3)': 20.0, 'V(2)': 16.66667, 'V(1)': 20.0}
{'V(3)': 5.714286, 'V(2)': 5.714286, 'V(1)': 20.0}
{'V(3)': 20.0, 'V(2)': 20.0, 'V(1)': 20.0}
{'V(3)': 13.33333, 'V(2)': 13.33333, 'V(1)': 20.0}
{'V(3)': 9.0, 'V(2)': 5.4, 'V(1)': 9.0}
{'V(3)': 9.0, 'V(2)': 9.0, 'V(1)': 9.0}
1 2 0.0
Generated Synthetic Data:
{'V(3)': 0.0, 'V(2)': 16.67, 'V(1)': 20.0}
{'V(3)': 2.3529

KeyboardInterrupt: 

In [ ]:
system_message = {
            "role": "system",
            "content": ("You are a helpful assistant designed to generate synthetic data. "
                        "Simulation Program with Integrated Circuit Emphasis (SPICE) is an essential software tool for electronics design that enables engineers to model and analyze the behavior of electronic circuits before they are physically constructed. "
                        "The user will provide a description of the circuit, and you need to respond with the SPICE code based on the description and the voltages derived from the SPICE simulation. "
                        "Explain your thought process for generating the SPICE code based on the description and the calculation process for getting the voltages."
                        "Provide the response in the following JSON format:\n"
                        "{\n"
                        "  \"description\": \"<description>\",\n"
                        " \"translation thought\": \"<translating to spice>\",\n"
                        "  \"spice\": \"<SPICE code>\",\n"
                        " \"calculation thought\": \"<computing the voltages>\",\n"
                        "  \"voltages\": <voltages>\n"
                        "}\n"
                        )
        }
def zero_shot(): # dumb
    messages = [system_message]
    return messages
def few_shots(examples): # denis' data without voltages
    messages = [system_message]
    for example in examples:
        messages.append({"role": "user", "content": f"Description: {example['description']}"})
        messages.append({
            "role": "assistant",
            "content": json.dumps({
                "description": example['description'],
                "spice": example['spice'],
            }, indent=4)
        })
    return messages
def augmented_data(examples): # with voltages
    messages = [system_message]
    for example in examples:
        messages.append({"role": "user", "content": f"Description: {example['description']}"})
        messages.append({
            "role": "assistant",
            "content": json.dumps({
                "description": example['description'],
                "spice": example['spice'],
                "voltages": example['voltages']
            }, indent=4)
        })
    return messages
def synthetic_data(examples, synthetic_examples): # with synthetic data
    messages = [system_message]
    for example in examples:
        messages.append({"role": "user", "content": f"Description: {example['description']}"})
        messages.append({
            "role": "assistant",
            "content": json.dumps({
                "description": example['description'],
                "spice": example['spice'],
                "voltages": example['voltages']
            }, indent=4)
        })
    for example in synthetic_examples:
        messages.append({"role": "user", "content": f"Description: {synthetic_examples['description']}"})
        messages.append({
            "role": "assistant",
            "content": json.dumps({
                "translation thought": synthetic_examples['translation thought'],
                "description": synthetic_examples['description'],
                "spice": synthetic_examples['spice'],
                "calculation thought": synthetic_examples['calculation thought'],
                "voltages": synthetic_examples['voltages']
            }, indent=4)
        })
    return messages
def inference(messages):
    response = client.chat.completions.create(
        model=datagen_model,
        messages=messages,
        max_tokens=500,
        n=1,
        stop=None,
        temperature=0.7
    )
    return response

In [ ]:
validation_dataset = []
for vdata in validation_dataset:
    # evaluate zero shot
    zero_shot_messages = zero_shot()
    zero_shot_messages.append({"role": "user", "content": f"Description: {vdata['description']}"})
    res = inference(zero_shot_messages)
    # evaluate few_shots
    few_shots_messages = few_shots()
    few_shots_messages.append({"role": "user", "content": f"Description: {vdata['description']}"})
    res = inference(few_shots_messages)
    # evaluate augmented_data
    augmented_messages = augmented_data()
    augmented_messages.append({"role": "user", "content": f"Description: {vdata['description']}"})
    res = inference(augmented_messages)
    # evaluate synthetic_data
    synthetic_messages = synthetic_data()
    synthetic_messages.append({"role": "user", "content": f"Description: {vdata['description']}"})
    res = inference(synthetic_messages)


In [17]:
new_examples = random.choices(synthetic_examples, k=5)


[{'description': 'Series RC circuit with DC voltage source.', 'spice': '\n* Series RC circuit\nV1 1 0 DC 10\nR1 1 2 1k\nC1 2 0 1u\n.op\n.print v(1) v(2)\n.end\n', 'voltages': {'V(2)': 10.0, 'V(1)': 10.0}}, {'description': 'Series RL circuit with DC current source.', 'spice': '\n* RL Circuit with Current Source\nI1 0 1 DC 2m\nR1 1 2 100\nL1 2 0 1m\n.op\n.print v(1) v(2)\n.end\n', 'voltages': {'V(2)': 0.0, 'V(1)': 0.2}}, {'description': 'Series RLC circuit with DC voltage source.', 'spice': '\n* RLC Circuit with Voltage Source\nV1 1 0 DC 5\nR1 1 2 500\nL1 2 3 100m\nC1 3 0 1u\n.op\n.print v(1) v(2) v(3)\n.end\n', 'voltages': {'V(3)': 5.0, 'V(2)': 5.0, 'V(1)': 5.0}}, {'description': 'Parallel RLC circuit with DC voltage source.', 'spice': '\n* Parallel RLC Circuit with Voltage Source\nV1 1 0 DC 12\nR1 1 2 2k\nL1 1 2 50m\nC1 1 2 2u\n.op\n.print v(1) v(2)\n.end\n', 'voltages': {'V(2)': 12.0, 'V(1)': 12.0}}, {'description': 'Series RLC circuit with DC current source.', 'spice': '\n* Series RL

In [21]:
for i in new_examples:
    volts = spice_sim(i)
    i["voltages"] = volts
    # del i['translation thought']
    # del i["calculation thought"]

with open("new_examples.txt", "w") as file:
        file.write(new_examples)

TypeError: write() argument must be str, not list